## Imports

In [1]:
import MDAnalysis as mda
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from MDAnalysis.lib.distances import distance_array
from MDAnalysis import units
from MDAnalysis.analysis import rdf

from openmm.unit import Quantity, Unit
from openmm.unit import kelvin, bar, litre, kilojoule_per_mole, mole, nanometer, angstrom, kilocalorie_per_mole
from openmm.unit import AVOGADRO_CONSTANT_NA, BOLTZMANN_CONSTANT_kB
R = BOLTZMANN_CONSTANT_kB * AVOGADRO_CONSTANT_NA # gas constant

import quantities
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from pymbar import timeseries
from MDAnalysis.analysis import density
from tqdm import tqdm
import os
import json


/tmp/ipykernel_40523/3626084654.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*             

## Set working directory

In [2]:
# ffs=input("Which force fields? (Options: JC, CHARMM27)")
# os.chdir("/home/bamo6610/Documents/GROMACS_files/MDAnalysis/SPW_ALL/%s"%ffs)

os.chdir("/home/bamo6610/Documents/GROMACS_files/MDAnalysis/SPW_ALL/JC")

## Loading data and creating atom group of ions

In [3]:
## 1 molal ##
u1m_o = mda.Universe('md_1m.gro', 'md_1m.xtc')
u1m_o.transfer_to_memory()
ions1m_o= u1m_o.select_atoms('resname SOD CLA')
densprof_1m_o="dens_o_1m.xvg"

u1m_r1 = mda.Universe('mdr1_1m.gro', 'mdr1_1m.xtc')
u1m_r1.transfer_to_memory()
ions1m_r1= u1m_r1.select_atoms('resname SOD CLA')
densprof_1m_r1="dens_r1_1m.xvg"

u1m_r2 = mda.Universe('mdr2_1m.gro', 'mdr2_1m.xtc')
u1m_r2.transfer_to_memory()
ions1m_r2= u1m_r2.select_atoms('resname SOD CLA')
densprof_1m_r2="dens_r2_1m.xvg"

u1m_r3 = mda.Universe('mdr3_1m.gro', 'mdr3_1m.xtc')
u1m_r3.transfer_to_memory()
ions1m_r3= u1m_r3.select_atoms('resname SOD CLA')
densprof_1m_r3="dens_r3_1m.xvg"

us_1m = [u1m_o,u1m_r1,u1m_r2,u1m_r3]
ions_1m = [ions1m_o,ions1m_r1,ions1m_r2,ions1m_r3]

# ## 2 molal ##
u2m_o = mda.Universe('md_2m.gro', 'md_2m.xtc')
u2m_o.transfer_to_memory()
ions2m_o= u2m_o.select_atoms('resname SOD CLA')
densprof_2m_o="dens_o_2m.xvg"

u2m_r1 = mda.Universe('mdr1_2m.gro', 'mdr1_2m.xtc')
u2m_r1.transfer_to_memory()
ions2m_r1= u2m_r1.select_atoms('resname SOD CLA')
densprof_2m_r1="dens_r1_2m.xvg"

u2m_r2 = mda.Universe('mdr2_2m.gro', 'mdr2_2m.xtc')
u2m_r2.transfer_to_memory()
ions2m_r2= u2m_r2.select_atoms('resname SOD CLA')
densprof_2m_r2="dens_r2_2m.xvg"

u2m_r3 = mda.Universe('mdr3_2m.gro', 'mdr3_2m.xtc')
u2m_r3.transfer_to_memory()
ions2m_r3= u2m_r3.select_atoms('resname SOD CLA')
densprof_2m_r3="dens_r3_2m.xvg"

us_2m = [u2m_o,u2m_r1,u2m_r2,u2m_r3]
ions_2m = [ions2m_o,ions2m_r1,ions2m_r2,ions2m_r3]

# ## 3 molal ##
u3m_o = mda.Universe('md_3m.gro', 'md_3m.xtc')
u3m_o.transfer_to_memory()
ions3m_o= u3m_o.select_atoms('resname SOD CLA')
densprof_3m_o="dens_o_3m.xvg"

u3m_r1 = mda.Universe('mdr1_3m.gro', 'mdr1_3m.xtc')
u3m_r1.transfer_to_memory()
ions3m_r1= u3m_r1.select_atoms('resname SOD CLA')
densprof_3m_r1="dens_r1_3m.xvg"

u3m_r2 = mda.Universe('mdr2_3m.gro', 'mdr2_3m.xtc')
u3m_r2.transfer_to_memory()
ions3m_r2= u3m_r2.select_atoms('resname SOD CLA')
densprof_3m_r2="dens_r2_3m.xvg"

u3m_r3 = mda.Universe('mdr3_3m.gro', 'mdr3_3m.xtc')
u3m_r3.transfer_to_memory()
ions3m_r3= u3m_r3.select_atoms('resname SOD CLA')
densprof_3m_r3="dens_r3_3m.xvg"

us_3m = [u3m_o,u3m_r1,u3m_r2,u3m_r3]
ions_3m = [ions3m_o,ions3m_r1,ions3m_r2,ions3m_r3]

## Utility functions

In [4]:
from typing import TypeAlias, TypeVar
Shape : TypeAlias = tuple
N = TypeVar('N')
M = TypeVar('M')

from openmm.unit import molar, kilogram, picoseconds
from dataclasses import dataclass

In [75]:
@dataclass
class Wall:
    '''For representing info about the location and orientation of a wall'''
    wall_pos : Quantity
    wall_normal : Quantity

@dataclass
class IonMolalInfo:
    molality : Quantity
    n_parts : int
    osm_coeff_exp : float
    molarity : Quantity = None

def osmotic_values(
        mean_force_wall : Quantity, 
        molarity : Quantity,
        cross_sectional_area : Quantity, 
        vant_hoff : int=2,
        T : Quantity=300*kelvin, 
        printvals : bool=False
    ) -> Quantity:
    '''Compute osmotic pressure and coefficient from mean force from wall'''
    osm_press=mean_force_wall/cross_sectional_area
    osm_bar = osm_press.in_units_of(bar / mole)
    osm_bar = osm_bar/AVOGADRO_CONSTANT_NA
    
    osm_press_ideal=vant_hoff*molarity*R*T
    osm_press_ideal=osm_press_ideal.in_units_of(bar)
    osm_coeff=osm_bar/osm_press_ideal
    
    if printvals:
        print(f"Osmotic Pressure Observed:", osm_bar)
        print(f"Osmotic Pressure Ideal:", osm_press_ideal)
        print(f"Osmotic Coefficient:", osm_coeff)

    return osm_bar, osm_coeff

def filter_coords_by_wall(
    coords_rel_wall : np.ndarray[Shape[N, M, 3], float], 
    wall_normal : np.ndarray[Shape[3], float],
    dist_unit : Unit=angstrom,
) -> np.ndarray[Shape[N, M, 3], float]:
    '''Zeros out all coordinates which are not in the normal direction relative to a wall'''
    in_right_direction = np.dot(coords_rel_wall, wall_normal) > 0.0
    return np.where(in_right_direction[..., np.newaxis], coords_rel_wall, 0.0) * dist_unit # need to broadcast direction bools, since dot product reduces dimension along xyz

def calculate_force_matrix_all_ions_rel_wall(
    coords : np.ndarray[Shape[N, M, 3], float], 
    wall_pos : np.ndarray[Shape[3], float],
    wall_normal : np.ndarray[Shape[3], float],
    K : Quantity,
    dist_unit : Unit=angstrom,
) -> np.ndarray[Shape[M], float]:
    '''Calculates net harmonic force experience by particle on one side of a wall
    coords should be a 3D array with axes of time, ion index, and spatial coordinate, respectively
    wall_pos is a vector describing the position of the wall in space
    wall_normal gives the normal direction relative to the wall (to determine whether particles behnd or ahead of the wall are kept)'''
    coords_rel_wall = coords - wall_pos
    coords_behind_wall = filter_coords_by_wall(coords_rel_wall, wall_normal=wall_normal, dist_unit=dist_unit)
    z_coords_rel_wall = coords_behind_wall[..., 2]

    # NOTE: cannot use np.abs, as this will discard unit info
    return K * abs(z_coords_rel_wall) # sum along time dimension to leave array with 1 force per particle


## Old functions

In [169]:
def calculate_net_force(z_values : np.ndarray[float], zwall : float, k : float) -> float:
    return k * abs(z_values - zwall).sum()

def get_universe_area_and_volume(u : mda.Universe) -> tuple[Quantity, Quantity]:
    '''Returns the XY-plane area and box volume of an MDAnalysis universe'''
    box_sizes = u.dimensions[:3] * angstrom
    box_x, box_y, box_z = box_sizes

    A_box = box_x * box_y
    V_box = A_box * box_z

    return A_box, V_box

def SPW_analysis(
        times : np.ndarray[1, Quantity],
        ion_coords : np.ndarray[Shape[N, M, 3], Quantity],
        molality_info : IonMolalInfo,
        k : Quantity,
        A_box : Quantity,
        walls : list[Wall],
        repnum : int,
        nskip : int=100,
    ) -> None:
    '''DOCS'''

    SPACER = f'\n{"-"*50}\n'

    print(SPACER)
    print(f"CONCENTRATION: {molality_info.molality}")

    #Specifying replicate number
    if repnum == 0:
        print("Data for original run")
    else:
        print("Data for replicate #",repnum)

    ## calculation of mean force matrix
    force_matrices = [
        calculate_force_matrix_all_ions_rel_wall(
            ion_coords,
            wall_pos=wall.wall_pos,
            wall_normal=wall.wall_normal,
            K=k,
            dist_unit=angstrom
        )    
        for wall in walls
     ]
    # print(
    #     [
    #         osmotic_values(fm.mean(axis=0).sum(), molarity=molality_info.molarity, cross_sectional_area=A_box)
    #         for fm in force_matrices
    #     ]
    # )
    if force_matrices:
        NULL_FORCE = 0.0 * force_matrices[0].unit # needed to get sum to start with the right units (can't just add to 0)

    force_matrix = sum(force_matrices, start=NULL_FORCE) / len(force_matrices) # NOTE: need to average over all walls, and numpy can't get the job done on its own
    forces_time_avg = force_matrix.mean(axis=0)
    forces_over_all_ions = force_matrix.sum(axis=1)

    ## calculation of autocorrelation function and plot of uncorrelated samples
    # statistical values of t0 (time after equil), g (statistical efficiency/correlation time of equil data), Neff (# of eff samples)
    t0, g, Neff_max = timeseries.detect_equilibration(forces_over_all_ions._value, nskip=nskip) # compute indices of uncorrelated timeseries
    forces_over_all_ions_equil = forces_over_all_ions[t0:]
    uncorr_sample_idxs = timeseries.subsample_correlated_data(forces_over_all_ions_equil, g=g)
    uncorr_sample_idxs = np.array(uncorr_sample_idxs)
    
    sample_forces = forces_over_all_ions_equil[uncorr_sample_idxs]
    sample_times  = times[uncorr_sample_idxs]

    # mean of full time series
    force_mean = sample_forces.mean()
    force_std  = sample_forces.std() / np.sqrt(Neff_max - 1) # TODO: make sure propagation of errors is being done correctly (since now the addition is being done up-front)

    ## calculation of osmotic values from statistical mean and std error
    print(f"FINAL STATISTICAL RESULTS:")
    osm_press_mean, osm_coeff_mean = osmotic_values(force_mean, molarity=molality_info.molarity, cross_sectional_area=A_box) 
    osm_press_std , osm_coeff_std  = osmotic_values(force_std , molarity=molality_info.molarity, cross_sectional_area=A_box)

    print(f"Osmotic Pressure = {osm_press_mean}+/-{osm_press_std**2}")
    print(f"\nOsmotic Coefficient = {osm_coeff_mean}+/-{osm_coeff_std**2}")
    print(SPACER)
    
    ################################################################################################
    # results = {
    #     'os_press_mean' : osm_press_mean,
    #     'os_press_std'  : osm_press_std ,
    #     'os_press_var'  : osm_press_std**2,
    #     'os_coeff_mean' : osm_coeff_mean,
    #     'os_coeff_std'  : osm_coeff_std ,
    #     'os_coeff_var'  : osm_coeff_std**2,
    # }
    results = dict()
    
    results['os_press_mean'] = osm_press_mean
    results['os_press_var'] = osm_press_std**2
    results['os_coeff_mean'] = osm_coeff_mean
    results['os_coeff_var'] = osm_coeff_std**2
    
    return results


## Computing releavnt physical quantities and running Osmotic calculations / stats

In [170]:
# define wall positions and ion conc parameters
walls = [
    Wall(
        wall_pos   =np.array([0.0, 0.0, 48.0]) * angstrom,
        wall_normal=np.array([0.0, 0.0, -1.0]) * angstrom,
    ),
    Wall(
        wall_pos   =np.array([0.0, 0.0, 96.0]) * angstrom,
        wall_normal=np.array([0.0, 0.0, 1.0]) * angstrom,
    ),
]

molal_infos = {
    1 : IonMolalInfo(
        molality=1.0*(mole / kilogram),
        n_parts=65,
        osm_coeff_exp=0.940,
        # molarity=0.98*molar
    ),
    2 : IonMolalInfo(
        molality=2.0*(mole / kilogram),
        n_parts=128,
        osm_coeff_exp=0.984,
        # molarity=1.92*molar
    ),
    3 : IonMolalInfo(
        molality=3.0*(mole / kilogram),
        n_parts=188,
        osm_coeff_exp=1.045,
        # molarity=2.82*molar
    ),
}

In [171]:
# pick your specific parameters here
# u = u1m_o
# atom_group = ions1m_o

# m_info = molal_infos[1]
# k = 4184 * kilojoule_per_mole / nanometer**2
def run_analysis(u,atom_group,concentration,repnum):
    m_info = molal_infos[concentration]
    k = 4184 * kilojoule_per_mole / nanometer**2
    # pre-processing parameters
    times = np.array([u.trajectory.time for _ in u.trajectory]) * picoseconds
    all_coords = u.trajectory.coordinate_array * angstrom
    ion_coords = all_coords[:, atom_group.indices, :]

    A_box, V_box = get_universe_area_and_volume(u)
    wall_sep = abs(walls[1].wall_pos - walls[0].wall_pos)[2] # !NEED TO EXTRACT Z-COMPONENT OF COORDINATE DIFFERENCE!
    V_mem = A_box * wall_sep
    m_info.molarity = m_info.n_parts / (V_mem * AVOGADRO_CONSTANT_NA)

    k = k.in_units_of(kilojoule_per_mole / angstrom**2)

    # run analysis
    results = SPW_analysis(times, ion_coords, m_info, k, A_box, walls, repnum)
    return results



In [198]:
results1m=list()
for i,(ui,ionsi) in enumerate(zip(us_1m,ions_1m)):
    results1m.append(run_analysis(ui,ionsi,1,i))


--------------------------------------------------

CONCENTRATION: 1.0 mol/kg
Data for original run
FINAL STATISTICAL RESULTS:
Osmotic Pressure = 47.993643005228954 bar+/-2.638304549708305 bar**2

Osmotic Coefficient = 0.9857335520328374+/-0.0011129523510645162

--------------------------------------------------


--------------------------------------------------

CONCENTRATION: 1.0 mol/kg
Data for replicate # 1
FINAL STATISTICAL RESULTS:
Osmotic Pressure = 51.20076738913496 bar+/-3.505204326105095 bar**2

Osmotic Coefficient = 1.0516041530708615+/-0.0014786486253573318

--------------------------------------------------


--------------------------------------------------

CONCENTRATION: 1.0 mol/kg
Data for replicate # 2
FINAL STATISTICAL RESULTS:
Osmotic Pressure = 52.35238924783529 bar+/-4.3513001758935355 bar**2

Osmotic Coefficient = 1.0752571253041143+/-0.0018355688927131554

--------------------------------------------------


--------------------------------------------------

In [199]:
results2m=list()
for i,(ui,ionsi) in enumerate(zip(us_2m,ions_2m)):
    results2m.append(run_analysis(ui,ionsi,2,i))


--------------------------------------------------

CONCENTRATION: 2.0 mol/kg
Data for original run
FINAL STATISTICAL RESULTS:
Osmotic Pressure = 107.31894213078989 bar+/-7.366346133130858 bar**2

Osmotic Coefficient = 1.119323424478904+/-0.0008013284105983443

--------------------------------------------------


--------------------------------------------------

CONCENTRATION: 2.0 mol/kg
Data for replicate # 1
FINAL STATISTICAL RESULTS:
Osmotic Pressure = 108.17463225049656 bar+/-9.516733043303987 bar**2

Osmotic Coefficient = 1.128248167642283+/-0.0010352525425571196

--------------------------------------------------


--------------------------------------------------

CONCENTRATION: 2.0 mol/kg
Data for replicate # 2
FINAL STATISTICAL RESULTS:
Osmotic Pressure = 106.04183080241086 bar+/-5.441988551274591 bar**2

Osmotic Coefficient = 1.1060033097150082+/-0.0005919922791401346

--------------------------------------------------


--------------------------------------------------


In [200]:
results3m=list()
for i,(ui,ionsi) in enumerate(zip(us_3m,ions_3m)):
    results3m.append(run_analysis(ui,ionsi,3,i))


--------------------------------------------------

CONCENTRATION: 3.0 mol/kg
Data for original run
FINAL STATISTICAL RESULTS:
Osmotic Pressure = 167.94907920045472 bar+/-15.105920168781074 bar**2

Osmotic Coefficient = 1.192638862663301+/-0.0007617454079363439

--------------------------------------------------


--------------------------------------------------

CONCENTRATION: 3.0 mol/kg
Data for replicate # 1
FINAL STATISTICAL RESULTS:
Osmotic Pressure = 165.05774363002368 bar+/-9.655950464740156 bar**2

Osmotic Coefficient = 1.1721069301114047+/-0.00048692008454921806

--------------------------------------------------


--------------------------------------------------

CONCENTRATION: 3.0 mol/kg
Data for replicate # 2
FINAL STATISTICAL RESULTS:
Osmotic Pressure = 155.26966638498195 bar+/-10.656032593982058 bar**2

Osmotic Coefficient = 1.1025999023339312+/-0.0005373511712355904

--------------------------------------------------


-----------------------------------------------

In [201]:
print(type(results1m))

<class 'list'>


In [202]:
for i in results3m:
    print(results3m[i])

TypeError: list indices must be integers or slices, not dict

### Stdev between replicates

In [210]:
def results_replicates(results,concentration):
    op_vals = list()
    oc_vals = list()
    op_errs = list()
    oc_errs = list()

    for r in results:
        v = r['os_press_mean']
        op_vals.append(v.value_in_unit(v.unit))
        v = r['os_press_var']
        op_errs.append(v.value_in_unit(v.unit))
        oc_vals.append(r['os_coeff_mean'])
        oc_errs.append(r['os_coeff_var'])
    oc_vals = np.array(oc_vals)
    op_vals = np.array(op_vals)
    op_errs = np.array(op_errs)
    oc_errs = np.array(oc_errs)

    print("----- Results standard deviation over replicates for %s molal -----"%concentration)
    stdop = op_vals.std(ddof=1)
    # print(f'std of osmotic presssure: {stdop:.3f}')
    stdoc = oc_vals.std(ddof=1)
    # print(f'std of osmotic coeff: {stdoc:.3f}')

    print("mean of osmotic pressure replicates: ",end="")
    mean_op=op_vals.mean()
    op_err=stdop/np.sqrt(len(op_vals))
    print(f'osmotic pressure {mean_op:.3f} +/- {op_err:.3f}')
    
    print("mean of osmotic coefficient replicates: ",end="")
    mean_oc=oc_vals.mean()
    oc_err=stdoc/np.sqrt(len(oc_vals))
    print(f'osmotic coeff {mean_oc:.3f} +/- {oc_err:.3f}')

    final_results = dict()

    final_results['mean_osmotic_pressure'] = mean_op
    final_results['uncertainity_osmotic_pressure'] = op_err
    
    final_results['mean_osmotic_coefficient'] = mean_oc
    final_results['uncertainity_osmotic_coefficient'] = oc_err

    return final_results


In [211]:
reps1m=results_replicates(results1m,1)
reps2m=results_replicates(results2m,2)
reps3m=results_replicates(results3m,3)

----- Results standard deviation over replicates for 1 molal -----
mean of osmotic pressure replicates: osmotic pressure 50.976 +/- 1.031
mean of osmotic coefficient replicates: osmotic coeff 1.047 +/- 0.021
----- Results standard deviation over replicates for 2 molal -----
mean of osmotic pressure replicates: osmotic pressure 106.851 +/- 0.547
mean of osmotic coefficient replicates: osmotic coeff 1.114 +/- 0.006
----- Results standard deviation over replicates for 3 molal -----
mean of osmotic pressure replicates: osmotic pressure 161.911 +/- 2.842
mean of osmotic coefficient replicates: osmotic coeff 1.150 +/- 0.020


## Write results to json file

In [205]:
# with open("final_results.txt","w") as file:
#     file.write("-- 1 molal replicate analysis --"+"\n")
#     file.write(f"Mean osmotic pressure: {reps1m['mean_osmotic_pressure']:.3f} +/- {reps1m['uncertainity_osmotic_pressure']:.3f} \n")
#     file.write(f"Mean osmotic coefficient: {reps1m['mean_osmotic_coefficient']:.3f} +/- {reps1m['uncertainity_osmotic_coefficient']:.3f} \n")

In [212]:
def format_dict(dictionary):
    newdict = {}
    for x in dictionary:
        if isinstance(dictionary[x],float):
            newdict[x] = round(dictionary[x],3)
        else:
            newdict[x] = dictionary[x]
    return newdict

In [213]:
json_1m=json.dumps(format_dict(reps1m),indent=4)
json_2m=json.dumps(format_dict(reps2m),indent=4)
json_3m=json.dumps(format_dict(reps3m),indent=4)

with open("final_results_1m.json","w") as outfile:
    outfile.write(json_1m)

with open("final_results_2m.json","w") as outfile:
    outfile.write(json_2m)

with open("final_results_3m.json","w") as outfile:
    outfile.write(json_3m)

## Density profiles using GROMACS

In [ ]:
def density_profile_conv(filename):
    x_zcoord,y=np.loadtxt(filename,comments=["@", "#"],unpack=True)
    y=(y/nanometer**3)/(AVOGADRO_CONSTANT_NA)
    y_conc=y.in_units_of(mole/litre)/2
    return x_zcoord,y_conc

### Density profile for 1m

In [ ]:
xo,yo=density_profile_conv(densprof_1m_o)
xr1,yr1=density_profile_conv(densprof_1m_r1)
xr2,yr2=density_profile_conv(densprof_1m_r2)
xr3,yr3=density_profile_conv(densprof_1m_r3)
plt.axvline(x = 4.8, color = 'gray', label = 'walls', linestyle = 'dashed')
plt.axvline(x = 9.6, color = 'gray', linestyle = 'dashed')
plt.axhline(y = 1, color = 'lightgray', linestyle = 'dashed')
plt.plot(xo,yo, c='purple', label='original')
plt.plot(xr1,yr1, c='g', label='rep 1')
plt.plot(xr2,yr2, c='c', label='rep 2')
plt.plot(xr3,yr3, c='m', label='rep 3')
plt.xlabel("Z coordinate (nm)")
plt.ylabel("Concentration (mol/L)")
plt.xticks(np.arange(0, 14.4, 1.0))
plt.yticks(np.arange(0, 1.1, 0.1))
plt.title('Concentration profile - JC - 1m')
plt.legend()
#plt.savefig("volume.png", format="png", dpi=300)
plt.show()

### Density profile for 2m

### Density profile for 3m

## Density profiles using MDanalysis

In [ ]:
def density_profile(universe, atom_group, bin_width=0.5, dim='z', method='atom', frameby=1):
    '''Calculate the partial density across the box'''

    if isinstance(atom_group, str): # if provided selection language, make AtomGroup
        ag = universe.select_atoms(atom_group)
    else: # else assume input is AtomGroup
        ag = atom_group

    print(f'\nCalculating the partial density profile of {atom_group} in the {dim} dimension...')

    dims = {'x': 0, 'y': 1, 'z': 2}
    d = dims[dim]
    box = universe.dimensions[d]

    n_bins = int(box / bin_width)
    bins = np.linspace(0, box, num=n_bins)

    counts = np.zeros(n_bins-1)
        
    if len(universe.trajectory) == 0:
        for b in tqdm(range(n_bins-1)):
            lb = bins[b]
            ub = bins[b+1]
            bin_atoms = universe.select_atoms(f'prop {dim} > {lb} and prop {dim} < {ub} and group ag', ag=ag)
            if method in ['atom', 'atoms', 'all']:
                counts[b] += len(bin_atoms)
            elif method in ['molecule', 'mol', 'residue', 'res']: 
                counts[b] += bin_atoms.n_residues
            elif method in ['mass', 'mass density']:
                box_dims = [box[i] for i in range(3) if i != d]
                dV = box_dims[0] * box_dims[1] * (ub-lb) * (10**-8)**3
                mass = bin_atoms.masses.sum() / 6.022 / 10**23
                counts[b] += mass / dV
    else:
        for ts in tqdm(universe.trajectory[::frameby]):
            for b in range(n_bins-1):
                lb = bins[b]
                ub = bins[b+1]
                bin_atoms = universe.select_atoms(f'prop {dim} > {lb} and prop {dim} < {ub} and group ag', ag=ag)

                if method in ['atom', 'atoms', 'all', 'number']:
                    counts[b] += len(bin_atoms)                 
                elif method in ['molecule', 'mol', 'residue', 'res']: 
                    counts[b] += bin_atoms.n_residues
                elif method in ['mass', 'mass density']:
                    box_dims = [box[i] for i in range(3) if i != d]
                    dV = box_dims[0] * box_dims[1] * (ub-lb) * (10**-8)**3
                    mass = bin_atoms.masses.sum() / 6.022 / 10**23
                    counts[b] += mass / dV

        counts = counts / len(universe.trajectory[::frameby])

    return bins, counts

In [ ]:
# nbins,ncounts=density_profile(u,ions,2)
# nbins1,ncounts1=density_profile(u1,ions1,2)
# nbins2,ncounts2=density_profile(u2,ions2,2)
# nbins3,ncounts3=density_profile(u3,ions3,2)

### Density plots

In [ ]:
nb=144.4/72
volbin=48*48*nb*angstrom**3
counts=(ncounts/AVOGADRO_CONSTANT_NA)/volbin.in_units_of(litre)/2
counts1=(ncounts1/AVOGADRO_CONSTANT_NA)/volbin.in_units_of(litre)/2
counts2=(ncounts2/AVOGADRO_CONSTANT_NA)/volbin.in_units_of(litre)/2
counts3=(ncounts3/AVOGADRO_CONSTANT_NA)/volbin.in_units_of(litre)/2

bins=nbins/10
bins1=nbins1/10
bins2=nbins2/10
bins3=nbins3/10

print(counts)

plt.axvline(x = 4.8, color = 'gray', label = 'walls', linestyle = 'dashed')
plt.axvline(x = 9.6, color = 'gray', linestyle = 'dashed')
plt.axhline(y = 1, color = 'lightgray', linestyle = 'dashed')
plt.plot(bins[1:],counts, c='orange', label='original')
plt.plot(bins1[1:],counts1, c='g', label='rep1')
plt.plot(bins2[1:],counts2, c='c', label='rep2')
plt.plot(bins3[1:],counts3, c='m', label='rep3')

plt.xlabel("Z coordinate (nm)")
plt.ylabel("Concentration (mol/L)")
plt.xticks(np.arange(0, 14.4, 1))
plt.title("Concentration profile of replicates - CHARMM27")
plt.legend()
plt.show()